In [ ]:
#Para limpiar el excel de turismo internacional de dataestur

import pandas as pd
import numpy as np

def parse_spanish_number(series):
    """
    Convierte números tipo '2.287.816,0' o '2287816,0' a float.
    """
    return (
        series.astype(str)
        .str.replace('.', '', regex=False)   # quita separador de miles
        .str.replace(',', '.', regex=False)  # cambia coma decimal por punto
        .replace({'nan': np.nan, 'None': np.nan, '': np.nan})
        .astype(float)
    )

def preparar_turismo_todos_a_bordo(
    path_excel: str,
    years_of_interest=None,   # si es None, coge todos los años
    summer_months=None,       # si es None, coge todos los meses
    output_path: str | None = None
) -> pd.DataFrame:
    # 1. Leer Excel
    df = pd.read_excel(path_excel)

    # 2. Solo filas de TOTAL (todas las procedencias)
    mask_total = (
        df["CONTINENTE_ORIGEN"].astype(str).str.strip().str.lower().eq("total")
        & df["PAIS_ORIGEN"].astype(str).str.strip().str.lower().eq("total")
    )
    base = df[mask_total].copy()

    # 3. Mapeo provincias Dataestur -> zonas de tu proyecto
    zone_config = {
        "Galicia": {
            "zona_proyecto": "A Coruña",
            "share": 0.40,   # % que asignas de Galicia a A Coruña
        },
        "Illes Balears": {
            "zona_proyecto": "Ibiza",
            "share": 0.30,   # % que asignas de Balears a Ibiza
        },
        "Málaga": {
            "zona_proyecto": "Málaga",
            "share": 1.00,   # Málaga ya es provincia, 100%
        },
    }

    # Nos quedamos solo con esas tres zonas
    base = base[base["PROVINCIA_DESTINO"].isin(zone_config)].copy()

    base["ZONA_PROYECTO"] = base["PROVINCIA_DESTINO"].map(
        lambda x: zone_config[x]["zona_proyecto"]
    )
    base["FACTOR_SHARE"] = base["PROVINCIA_DESTINO"].map(
        lambda x: zone_config[x]["share"]
    )

    # 4. Limpiar numéricos
    base["TURISTAS"] = parse_spanish_number(base["TURISTAS"])
    base["PERNOCTACIONES"] = parse_spanish_number(base["PERNOCTACIONES"])
    base["ESTANCIA_MEDIA"] = parse_spanish_number(base["ESTANCIA_MEDIA"])

    # 5. Turistas ajustados a tu zona (A Coruña / Ibiza / Málaga)
    base["TURISTAS_AJUSTADOS"] = (
        base["TURISTAS"] * base["FACTOR_SHARE"]
    ).round(0).astype("Int64")

    # 6. Filtros opcionales (años y meses)
    if years_of_interest is not None:
        base = base[base["AÑO"].isin(years_of_interest)]

    if summer_months is not None:
        base = base[base["MES"].isin(summer_months)]

    # 7. Ordenar
    base = base.sort_values(["ZONA_PROYECTO", "AÑO", "MES"])

    # 8. Guardar
    if output_path:
        base.to_excel(output_path, index=False)

    return base


if __name__ == "__main__":
    # Ruta a tu fichero original
    input_path = r"Turismo_int_ dataestur.xlsx"   # ajusta si está en otra carpeta

    # Fichero de salida ya limpio SOLO JULIO todos los años
    output_path = r"turistas_internacionales_julio_clean.xlsx"

    # ✅ TODOS los años, SOLO julio (MES = 7)
    df_turismo = preparar_turismo_todos_a_bordo(
        path_excel=input_path,
        years_of_interest=None,   # todos los años
        summer_months=[7],        # solo julio
        output_path=output_path,
    )

    print(df_turismo.head())
    print("Filas finales (solo julio):", len(df_turismo))


       AÑO  MES PROVINCIA_DESTINO CONTINENTE_ORIGEN PAIS_ORIGEN   TURISTAS  \
72    2023    7           Galicia             Total       Total   249724.0   
1100  2024    7           Galicia             Total       Total   260182.0   
2269  2025    7           Galicia             Total       Total   295393.0   
5173  2023    7     Illes Balears             Total       Total  2287816.0   
6485  2024    7     Illes Balears             Total       Total  2428342.0   

      PERNOCTACIONES  ESTANCIA_MEDIA  \
72         1734132.0             6.9   
1100       1801002.0             6.9   
2269       2229140.0             7.5   
5173      14695616.0             6.4   
6485      15587112.0             6.4   

                                   archivo_origen ZONA_PROYECTO  FACTOR_SHARE  \
72             INT_turismo_receptor_ccaa_pais.csv      A Coruña           0.4   
1100           INT_turismo_receptor_ccaa_pais.csv      A Coruña           0.4   
2269           INT_turismo_receptor_ccaa_pais.c

In [5]:
media_por_zona = (
    df_turismo
    .groupby("ZONA_PROYECTO", as_index=False)["TURISTAS_AJUSTADOS"]
    .mean()
    .rename(columns={"TURISTAS_AJUSTADOS": "TURISTAS_MEDIOS"})
)
print(media_por_zona)


  ZONA_PROYECTO  TURISTAS_MEDIOS
0      A Coruña    107373.333333
1         Ibiza    704479.666667
2        Málaga    791564.666667


In [ ]:
#Para limpiar el excel de turismo nacional de dataestur

import pandas as pd
import numpy as np

def parse_spanish_number(series: pd.Series) -> pd.Series:
    """
    Convierte números tipo '2.287.816,0' o '2287816,0' a float.
    Si algo no se puede convertir, lo pasa a NaN.
    """
    s = (
        series.astype(str)
        .str.strip()
        .str.replace('.', '', regex=False)   # quita separador de miles
        .str.replace(',', '.', regex=False)  # coma decimal -> punto
        .replace({'nan': np.nan, 'None': np.nan, '': np.nan, '-': np.nan})
    )
    return pd.to_numeric(s, errors="coerce")


def preparar_turismo_nacional_todos_a_bordo(
    path_excel,
    years_of_interest=None,     # si es None, coge todos los años
    summer_months=None,         # si es None, coge todos los meses
    output_path=None
) -> pd.DataFrame:
    # 1. Leer Excel
    df = pd.read_excel(path_excel)

    # 2. QUEDARNOS SOLO CON EL TOTAL NACIONAL (CCAA_ORIGEN = 'Total Nacional')
    mask_total_nac = (
        df["CCAA_ORIGEN"]
        .astype(str)
        .str.strip()
        .str.lower()
        .eq("total nacional")
    )
    base = df[mask_total_nac].copy()

    # 3. Filtrar por provincias destino del proyecto:
    #    A Coruña, Illes Balears, Málaga
    provincias_objetivo = ["A Coruña", "Illes Balears", "Málaga"]
    base = base[base["PROVINCIA_DESTINO"].isin(provincias_objetivo)].copy()

    # 4. Crear ZONA_PROYECTO y FACTOR_SHARE
    #    - A Coruña  -> A Coruña (100%)
    #    - Málaga    -> Málaga (100%)
    #    - Illes Balears -> Ibiza (30% del total Balears)
    def map_zona_proyecto(prov):
        if prov == "Illes Balears":
            return "Ibiza"
        else:
            return prov  # A Coruña y Málaga se quedan igual

    def map_share(prov):
        if prov == "Illes Balears":
            return 0.30   # 30% de Illes Balears se atribuye a Ibiza
        else:
            return 1.00   # A Coruña y Málaga 100%

    base["ZONA_PROYECTO"] = base["PROVINCIA_DESTINO"].map(map_zona_proyecto)
    base["FACTOR_SHARE"] = base["PROVINCIA_DESTINO"].map(map_share).astype(float)

    # 5. Limpiar numéricos (aseguramos que son float)
    base["TURISTAS"] = parse_spanish_number(base["TURISTAS"])
    base["PERNOCTACIONES"] = parse_spanish_number(base["PERNOCTACIONES"])
    base["ESTANCIA_MEDIA"] = parse_spanish_number(base["ESTANCIA_MEDIA"])

    # 6. Calcular TURISTAS_AJUSTADOS (ya asociados a A Coruña / Ibiza / Málaga)
    base["TURISTAS_AJUSTADOS"] = base["TURISTAS"] * base["FACTOR_SHARE"]
    base["TURISTAS_AJUSTADOS"] = (
        base["TURISTAS_AJUSTADOS"]
        .round(0)
        .astype("Int64")    # permite NaN
    )

    # 7. Filtros opcionales de año y mes
    if years_of_interest is not None:
        base = base[base["AÑO"].isin(years_of_interest)]

    if summer_months is not None:
        base = base[base["MES"].isin(summer_months)]

    # 8. Ordenar
    base = base.sort_values(["ZONA_PROYECTO", "AÑO", "MES"])

    # 9. Guardar si hace falta
    if output_path:
        base.to_excel(output_path, index=False)

    return base


if __name__ == "__main__":
    # Ruta a tu fichero original de turismo nacional
    input_path = r"Turismo_nac_ dataestur.xlsx"   # ajusta si está en otra carpeta

    # Fichero de salida: solo JULIO todos los años
    output_clean_julio = r"turistas_nacional_julio_clean.xlsx"

    # ✅ TODOS los años, SOLO julio (MES = 7)
    df_turismo_nac = preparar_turismo_nacional_todos_a_bordo(
        path_excel=input_path,
        years_of_interest=None,   # todos los años
        summer_months=[7],        # solo julio
        output_path=output_clean_julio,
    )

    print("Preview datos limpios (julio, todos los años):")
    print(df_turismo_nac.head())
    print("Filas finales turismo nacional (solo julio):", len(df_turismo_nac))

    # 10. MEDIA DE TURISTAS NACIONALES EN JULIO POR ZONA_PROYECTO
    media_nacional = (
        df_turismo_nac
        .groupby("ZONA_PROYECTO", as_index=False)["TURISTAS_AJUSTADOS"]
        .mean()
        .rename(columns={"TURISTAS_AJUSTADOS": "TURISTAS_NAC_MEDIOS_JULIO"})
    )

    print("\nMedia de turistas nacionales en julio por zona:")
    print(media_nacional)

    # Guardamos también la media en otro Excel (opcional)
    output_media = r"turistas_nacional_julio_media_por_zona.xlsx"
    media_nacional.to_excel(output_media, index=False)


Preview datos limpios (julio, todos los años):
       AÑO  MES     CCAA_ORIGEN PROVINCIA_ORIGEN   CCAA_DESTINO  \
199   2023    7  Total Nacional   Total Nacional        Galicia   
2632  2024    7  Total Nacional   Total Nacional        Galicia   
5057  2025    7  Total Nacional   Total Nacional        Galicia   
5112  2023    7  Total Nacional   Total Nacional  Illes Balears   
5733  2024    7  Total Nacional   Total Nacional  Illes Balears   

     PROVINCIA_DESTINO  TURISTAS  PERNOCTACIONES  ESTANCIA_MEDIA  \
199           A Coruña  440970.0       2209059.0             5.0   
2632          A Coruña  419676.0       2429446.0             5.8   
5057          A Coruña  328560.0       1457152.0             4.4   
5112     Illes Balears  475218.0       2974383.0             6.3   
5733     Illes Balears  417104.0       2834846.0             6.8   

                                        archivo_origen ZONA_PROYECTO  \
199              NAC_turismo_interno_prov_ccaa (5).csv      A Coruña 

In [13]:
# 📌 MEDIA TURISTAS NACIONALES (julio, todos los años)
media_nac_por_zona = (
    df_turismo_nac
    .groupby("ZONA_PROYECTO", as_index=False)["TURISTAS_AJUSTADOS"]
    .mean()
    .rename(columns={"TURISTAS_AJUSTADOS": "TURISTAS_NAC_MEDIOS_JULIO"})
)
print("\nMedia turistas NACIONALES en julio por zona:")
print(media_nac_por_zona)




Media turistas NACIONALES en julio por zona:
  ZONA_PROYECTO  TURISTAS_NAC_MEDIOS_JULIO
0      A Coruña                   396402.0
1         Ibiza                   121146.0
2        Málaga              863743.666667
